<a href="https://colab.research.google.com/github/warut/kaggle/blob/LLMs-U-Cant-Pls-Em-All/Gemma_2_Baseline_Generating_Essays_w_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

llms_you_cant_please_them_all_path = kagglehub.competition_download('llms-you-cant-please-them-all')
nischaydnk_immutabledict_path = kagglehub.dataset_download('nischaydnk/immutabledict')
fernandosr85_sentencepiece_0_2_0_cp310_cp310_manylinux_path = kagglehub.dataset_download('fernandosr85/sentencepiece-0-2-0-cp310-cp310-manylinux')
nischaydnk_gemma_pytorch_path = kagglehub.dataset_download('nischaydnk/gemma-pytorch')
google_gemma_2_pytorch_gemma_2_2b_it_1_path = kagglehub.model_download('google/gemma-2/PyTorch/gemma-2-2b-it/1')

print('Data source import complete.')


# Gemma 2 Baseline PyTorch
## Generate Random Essays for given Topics with Gemma Models

##### This is a baseline notebook in which we will be using Gemma 2 models inference in PyTorch for generating essays.
##### You can check out the Github repo of the official PyTorch implementation [here](https://github.com/google/gemma_pytorch).

##### You can also use other Gemma Variants including 9B, 27B parameter models. You just need to add different variant from Kaggle models and update the parameters "GEMMA_MODEL" , "MODEL_CONFIG" & "MODEL_DIR".

### Note: Since the competition is more than just generating an essay for a given topic, this notebook is just to provide a solution with the help of LLMs.

## 1.  Installing additional dependencies

In [ ]:
!pip install --no-index --no-deps /kaggle/input/immutabledict/immutabledict-4.1.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/sentencepiece-0-2-0-cp310-cp310-manylinux/sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

!mkdir /kaggle/working/gemma/
!cp /kaggle/input/gemma-pytorch/gemma_pytorch-main/gemma/* /kaggle/working/gemma/

In [ ]:
import sys
sys.path.append("/kaggle/working/")

# 2.  Import Gemma Modules

In [ ]:
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
from transformers import AutoTokenizer
import contextlib
import os
import torch
import random
import pandas as pd
random.seed(0)


# 3. Select Gemma Model Variant

In [ ]:
GEMMA_MODEL = '2b-it'
DEVICE = 'cuda'
MODEL_CONFIG = '2b-v2'
MODEL_DIR = "/kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1"
CKPT_PATH = os.path.join(MODEL_DIR, f'model.ckpt')
TOKENIZER_PATH = os.path.join(MODEL_DIR, f'tokenizer.model')


# 4. Loading Model & Model Config

In [ ]:
# Set up model config.
CONFIG = get_model_config(MODEL_CONFIG)
CONFIG.quant = 'quant' in GEMMA_MODEL
CONFIG.tokenizer = TOKENIZER_PATH
torch.set_default_dtype(CONFIG.get_dtype())

In [ ]:
# Initiialize the model and load the weights.
device = torch.device(DEVICE)
model = GemmaForCausalLM(CONFIG)
model.load_weights(CKPT_PATH)
model = model.to(device).eval()
model.config.use_cache = False
model.config.pretraining_tp = 1

# 5. Defining Gemma Chat Template

In [ ]:
# This is the prompt format the model expects
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

# 6. Sample Generation

In [ ]:
max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, max_seq_length=max_seq_length)
EOS_TOKEN = tokenizer.eos_token

input_text = "What is Kaggle?"

In [ ]:
input_text

In [ ]:
print('Chat prompt:\n', USER_CHAT_TEMPLATE.format(prompt=input_text))

results = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=input_text),
    device=DEVICE,
    output_len=128,
)
print(results)

# 7. Design your own Prompt

In [ ]:
prompt_for_llm = (
    "<start_of_turn>user\nGenerate an essay for the following topic with no more than 100 words: {topic_name}."
    "Pay attention to detail, clarity, and overall quality in your generated essay."
    "<end_of_turn>\n<start_of_turn>model\n"
)


In [ ]:
test = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')

In [ ]:
test

In [ ]:
sample_sub = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/sample_submission.csv')

In [ ]:
test.loc[0,'topic']

In [ ]:
sample_sub

# 7. Basic Baseline
#### Here we will try to use Gemma model for generating essays relevant to the provided topics in test csv.

In [ ]:


predictions = []

for i in range(len(test)):
    topic = test.loc[i, 'topic']
    gen_essay = model.generate(
        prompt_for_llm.format(topic_name=topic),
        device=device,
        output_len=128,
    )
    predictions.append(gen_essay)

    if i<=2:
        print('Topic: ', topic)
        print('Generated Essay: ', gen_essay)
        print('\n\n***********************\n\n')

In [ ]:
predictions[0]

In [ ]:
sample_sub['essay'] = predictions


In [ ]:
sample_sub

In [ ]:
sample_sub.to_csv('submission.csv',index=False)